<h2>CS 4780/5780 Final Project </h2>
<h3>Election Result Prediction for US Counties</h3>

Harry Dang (hd259)

Blake Gallay (bjg222)

Sawyer Huang (smh367)

<h3>Introduction:</h3>

<p> The final project is about conducting a real-world machine learning project on your own, with everything that is involved. Unlike in the programming projects 1-5, where we gave you all the scaffolding and you just filled in the blanks, you now start from scratch. The programming project provide templates for how to do this, and the most recent video lectures summarize some of the tricks you will need (e.g. feature normalization, feature construction). So, this final project brings realism to how you will use machine learning in the real world.  </p>

The task you will work on is forecasting election results. Economic and sociological factors have been widely used when making predictions on the voting results of US elections. Economic and sociological factors vary a lot among counties in the United States. In addition, as you may observe from the election map of recent elections, neighbor counties show similar patterns in terms of the voting results. In this project you will bring the power of machine learning to make predictions for the county-level election results using Economic and sociological factors and the geographic structure of US counties. </p>
<p>

<h3>Your Task:</h3>
Plase read the project description PDF file carefully and make sure you write your code and answers to all the questions in this Jupyter Notebook. Your answers to the questions are a large portion of your grade for this final project. Please import the packages in this notebook and cite any references you used as mentioned in the project description. You need to print this entire Jupyter Notebook as a PDF file and submit to Gradescope and also submit the ipynb runnable version to Canvas for us to run.

<h3>Due Date:</h3>
The final project dataset and template jupyter notebook will be due on <strong>December 15th</strong> . Note that <strong>no late submissions will be accepted</strong>  and you cannot use any of your unused slip days before.
</p>

<h2>Part 1: Basics</h2><p>

<h3>1.1 Import:</h3><p>
Please import necessary packages to use. Note that learning and using packages are recommended but not required for this project. Some official tutorial for suggested packacges includes:
    
https://scikit-learn.org/stable/tutorial/basic/tutorial.html
    
https://pytorch.org/tutorials/
    
https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html
<p>

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import sklearn

from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge

# PyTorch, used for neural network
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

<h3>1.2 Weighted Accuracy:</h3><p>
Since our dataset labels are heavily biased, you need to use the following function to compute weighted accuracy throughout your training and validation process and we use this for testing on Kaggle.
<p>

In [ ]:
def weighted_accuracy(pred, true):
    assert(len(pred) == len(true))
    num_labels = len(true)
    num_pos = sum(true)
    num_neg = num_labels - num_pos
    frac_pos = num_pos/num_labels
    weight_pos = 1/frac_pos
    weight_neg = 1/(1-frac_pos)
    num_pos_correct = 0
    num_neg_correct = 0
    for pred_i, true_i in zip(pred, true):
        num_pos_correct += (pred_i == true_i and true_i == 1)
        num_neg_correct += (pred_i == true_i and true_i == 0)
    weighted_accuracy = ((weight_pos * num_pos_correct) 
                         + (weight_neg * num_neg_correct))/((weight_pos * num_pos) + (weight_neg * num_neg))
    return weighted_accuracy

<h2>Part 2: Baseline Solution</h2><p>
Note that your code should be commented well and in part 2.4 you can refer to your comments. (e.g. # Here is SVM, 
# Here is validation for SVM, etc). Also, we recommend that you do not to use 2012 dataset and the graph dataset to reach the baseline accuracy for 68% in this part, a basic solution with only 2016 dataset and reasonable model selection will be enough, it will be great if you explore thee graph and possibly 2012 dataset in Part 3.

<h3>2.1 Preprocessing and Feature Extraction:</h3><p>
Given the training dataset and graph information, you need to correctly preprocess the dataset (e.g. feature normalization). For baseline solution in this part, you might not need to introduce extra features to reach the baseline test accuracy.
<p>

In [ ]:
trainfile = 'train_2016.csv'
testfile = 'test_2016_no_label.csv'

In [ ]:
"""
Functions for processing data
"""

def clean(df):
    new_header = df.iloc[0] # first row is the header names for some reason
    df = df[1:] # remove first row
    df.columns = new_header # rename columns
    return df
    
def preprocess(file, train, creative=False):
    """
    Returns a processed dataframe from the csv file
    file: path to csv file
    train: True if contains DOM/GOP column, False otherwise
    """
    df = clean(pd.read_csv(file, sep=',',header=None, encoding='unicode_escape'))

    # convert string to int
    if train:
        df[['DEM', 'GOP']] = df[['DEM', 'GOP']].astype(int)
    df['MedianIncome'] = df['MedianIncome'].str.replace(',','').astype(int)
    df[['MigraRate', 'BirthRate', 'DeathRate', 'BachelorRate', 'UnemploymentRate']] = df[['MigraRate', 'BirthRate', 'DeathRate', 'BachelorRate', 'UnemploymentRate']].astype(float)

    # create percentage of DEM votes, add column 'Label'
    if train:
        label = df['DEM'] / (df['DEM'] + df['GOP'])
        df.insert(2, 'DEM%', label, True)

        df['Label'] = df.apply(lambda row: int(row['DEM'] > row['GOP']), axis=1)
    return df

def normalize(df, columns, norm):
    """
    Normalizes columns in the given df dataframe
    norm = 'minmax' or 'std'
    """
    if norm == 'minmax':
        df[columns] = df[columns].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
    elif norm == 'std':
        df[columns] = df[columns].apply(lambda x: (x - x.mean()) / x.std())
        
    return df

def split(df, proportion=0.8, norm='std'):
    """
    splits df into a training and test set
    normalizes both sets separately
    DO NOT NORMALIZE BEFORE
    """
    xTr = df.sample(frac=proportion) # split dataset
    xTe = df.drop(xTr.index)
    features = ['MedianIncome', 'MigraRate', 'BirthRate', 'DeathRate', 'BachelorRate', 'UnemploymentRate', ]
    normalize(xTr, features, norm) # normalize individually, no data leak
    normalize(xTe, features, norm)
    return xTr, xTe

<h3>2.2 Use At Least Two Training Algorithms from class:</h3><p>
You need to use at least two training algorithms from class. You can use your code from previous projects or any packages you imported in part 1.1.
    
<h4> combined with </h4>
    
<h3>2.3 Training, Validation and Model Selection:</h3><p>
You need to split your data to a training set and validation set or performing a cross-validation for model selection.

<h4> Soft-Margin SVM </h4>

In [ ]:
df = preprocess(trainfile, True)

# Randomly shuffle the order of the instances, so that in our analysis we do not
# accidentally fit to an arbitrary ordering. This matters when we split into
# different sets by row indices
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
def predict(clf,X):
    preds = []
    for row in X:
        pred = clf.predict([row])
        preds.append(pred[0])
    preds = np.array(preds)
    return preds

def get_svm(kernel,C):
    svm_clf = svm.SVC(kernel=kernel,C=C)
    
    def fit(xTr,yTr):
        svm_clf.fit(xTr,yTr)
        
        return lambda X: predict(svm_clf,X)
    
    return fit
    
def train_svm(df, features, k, normfeats, label, cs, kernels):
    
    # Group size
    size = int(np.floor(df.shape[0] / k))-1

    # We keep track of the best result so far: best = [accuracy,model]
    best = [0,None]
    
    for C in cs:
        for kernel in kernels:

            # Accuracies on each subset when the SVM is trained on the remaining instances
            accs = []

            for i in range(k):

                trainset = normalize(df.drop(df.index[(i*size):(i+1)*size]).copy(), normfeats, 'std')

                valset = normalize(df[i*size:(i+1)*size].copy(), normfeats, 'std')

                xTr = trainset[features].to_numpy()
                yTr = trainset[label].to_numpy()

                yVa = valset[label].to_numpy()
                xVa = valset[features].to_numpy()

                model = get_svm(kernel,C)
                trained_model = model(xTr,yTr)
                preds = trained_model(xVa)

                acc = weighted_accuracy(preds, yVa)
                accs.append(acc)

            cross_val_acc = np.average(accs)
            if(cross_val_acc > best[0]):
                best = [cross_val_acc,model,kernel,C]
                
    return best

In [ ]:
# Values of C for SVM
cs = np.logspace(0,2,num=10)

# Available kernels for SVM: linear, poly, rbf, sigmoid
kernels = ['rbf','poly']

# Features we train the SVM on
features = ['MedianIncome', 'MigraRate', 'BirthRate', 'DeathRate', 'BachelorRate', 'UnemploymentRate']
normfeats = ['MedianIncome', 'MigraRate', 'BirthRate', 'DeathRate', 'BachelorRate', 'UnemploymentRate']
label = 'Label'

svm_results = train_svm(df, features, 10, normfeats, label, cs, kernels)

model = svm_results[1]

In [ ]:
print('(Basic SVM)')
print('k-fold cross-val accuracy: ' + str(svm_results[0]))
print('best kernel: ' + svm_results[2])
print('best C: ' + str(svm_results[3]))

In [ ]:
all_feats = df[features].copy()
normalize(all_feats, features, 'std')
all_feats = all_feats.to_numpy()
all_labels = df[label].to_numpy()

trained_model = model(all_feats,all_labels)
preds = trained_model(all_feats)

acc = weighted_accuracy(preds, all_labels)

print('SVM training accuracy: ' + str(acc))

<h4> Neural Network </h4>

In [ ]:
# Modified from the discussion notebook
class CSVDataset(Dataset):
    def __init__(self, dataset, train):
        # Where the initial logic happens like reading a csv, doing data augmentation, etc.
        self.dataset = dataset   # df is a Pandas dataframe in the above format
        self.train = train  # train is True when there are DOM/GOP columns, False otherwise

    def __len__(self):
        # Returns count of samples (an integer) you have. 
        return len(self.dataset.index)

    def __getitem__(self, idx):
        # Given an index, returns the correponding datapoint. 
        # This function is called from dataloader like this:
        # img, label = CSVDataset.__getitem__(99)  # For 99th item
        # sol = 'basic' when using basic solution, 'creative' when using creative
        result = self.dataset.iloc[idx]
        features = ['MedianIncome', 'MigraRate', 'BirthRate', 'DeathRate', 'BachelorRate', 'UnemploymentRate']    # old features, no states
        if self.train:
            return (torch.Tensor(result[features]), torch.as_tensor(result['Label']))
        else:
            return torch.Tensor(result[features])

# Modified from the discussion notebook
class Net(nn.Module):
    def __init__(self):
        """
        You need to initialize most NN layers here.
        """
        super(Net, self).__init__()
        # change first number to match inputs
        # according to resources, hidden layer <= 2 * input
        self.fc1 = nn.Linear(6, 12)
        self.fc2 = nn.Linear(12, 12) # hidden layer
        self.fc3 = nn.Linear(12, 1)

    def forward(self, x):
        """
        Define in what order the input x is forwarded through all the NN layers to become a final output. 
        """
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

In [ ]:
def trainNN(xTr, epochs=10, lr = 0.06):
    """
   returns new neural networks after epochs with a learning rate of lr
    """
    net = Net()     # new net
    criterion = nn.MSELoss()    # loss function
    optimizer = optim.SGD(net.parameters(), lr = lr)    # optimization
    dataset = CSVDataset(xTr, True)     # dataset for PyTorth
    dataloader = DataLoader(dataset, shuffle = True) # shuffle data
    networks = []   # all networks
    accuracies = []     # all accuracies
    for epoch in range(epochs):
        preds = []
        real = []
        for batch_idx, (input, target) in enumerate(dataloader):
            optimizer.zero_grad() # resets gradient
            output = net(input) # output for current input
            loss = criterion(output, target.float()) # calculate loss
            loss.backward() # backprop?
            optimizer.step()
            prediction = output.item() # prediction with current input
            preds += [1 if prediction >= 0.2 else 0]   # 0/1 prediction with threshold
            real += [target.item()]    # real label
        accuracy = weighted_accuracy(preds, real) # calculate accuracy
        networks += [net]
        accuracies += [accuracy]
        print("epoch = " + str(epoch) + ", loss = " + str(loss)) # loss for current epoch
        print(accuracy) # prints out Tensor?? not sure why, should be a float
    net = networks[np.argmax(accuracies)] # return best network based on training accuracy
    return net

def predictNN(net, xTe):
    preds = []
    real = []
    raw = []
    dataset = CSVDataset(xTe, True) # dataset for PyTorch
    dataloader = DataLoader(dataset, shuffle = False)
    for x, y in dataloader:
        prediction = net(x).item()  # prediction with network
        raw += [prediction] # raw predictions
        prediction = 1 if prediction >= 0.2 else 0 # 0/1 prediction with threshold
        y = y.item()    # real label
        preds += [prediction]
        real += [y]
    print(preds) # all 0/1 predictions
    print(real) # all real labels
    print(raw) # all raw predictions
    return weighted_accuracy(preds, real)

In [ ]:
df = preprocess(trainfile, True) # create Pandas dataframe
xTr, xTe = split(df, norm = 'minmax') # split into training and test
net = trainNN(xTr, 25, 0.0075) # train network 
accuracy = predictNN(net, xTe) # calculate validation accuracy
print(accuracy)

<h3>2.4 Explanation in Words:</h3><p>

<h4> 2.4.1 How did you preprocess the dataset and features? </h4> <p>

Our preprocessing included normalization of the existing features, and extraction of new features which we later show to be informative and improve our models’ accuracy. 

We initially chose min/max feature normalization over standardization because we found that for one of our models standardization led to significant overfitting. We hypothesized that this is because providing features which have been standardized over the entire training set to algorithms which should only be privy to subsets of the data (e.g. when using a training/validation/test split, or k-fold cross-validation) leaks information about the mean and standard deviation of each feature over the whole training set. This can lead to poor generalization error, as the algorithms have indirectly taken information about the training set as input. However, we eventually realized that this issue can be avoided by postponing standardization until the data is split, whether that be into standalone test/validation sets, or into different training and validation sets during k-fold cross-validation. This resulted ultimately in the best accuracy for both of our models. 

In the basic step we also added ‘percentile bins’ for only the MedianIncome feature, as we note that there are several important thresholds, i.e. the poverty line, the top 1%, etc, which may be more significant than specific dollar figures. 

Finally we add a ‘Population Change’ feature, which is equal to MigraRate + BirthRate - DeathRate. We add this feature because we note that all three of these features say something about the way that the population is changing, and we can get an estimate of the overall trend, i.e. population growth or decline, by combining them into a single additional feature. 

We add more features in the creative part of the project.


<h4> 2.4.2 Which two learning methods from class did you choose and why did you made the choices? </h4> <p>
    
We have two strong models that were developed largely independently, but we have recently sought to combine. 

The first is a soft-margin SVM. We decided to use an SVM initially because it is simple to implement, has parameters which are easy to understand intuitively, and does not depend on strong assumptions about the data. Specifically, it does not assume that data are i.i.d., which we suspect they are not because after watching election results come in every four years we tend to notice that results are strongly correlated across states and counties, and this means that final results can be projected quite early on in vote counts. For example if California goes red then that probably means bad news for the Democrats across the board. And even though vote counts are quantitative, ultimately we are predicting a binary label and so an SVM will work fine. 

The second is a neural network. A neural network was chosen, because they are able to express complex functions so it can classify data that may or may not be linearly separable. Due to the complex nature of real world voting habits, there can be many complex relationships that simpler learning methods would not be able to learn from. Additionally, it can better generalize at unsee data since it is learning from relationships between the features.


<h4> 2.4.3 How did you do the model selection? </h4> <p>
    
To select the best SVM, we use k-fold cross validation. SVMs take two main parameters: the regularization parameter C and the kernel K. We quickly identified the rbf kernel as the best performing kernel. The best value of C varied as we made adjustments to the model, but generally we provided a range in logspace between 10^0 and 10^2, and the best value tended to come to between 20 and 70. We stuck with k=10 for the cross-validation. 

To select the best neural network, we first started off using the sample neural network that was demonstrated in discussion. This was modified to fit our given dataset. First, the shape of the network was modified to have 6 input nodes and a single hidden layer with 12 nodes. After printing the accuracy for each training epoch, the learning rate was manually adjusted. This was done a few times until the validation accuracy was able to surpass the needed accuracy. It was manually adjusted instead of using a procedural method such as k-fold, because the training is inconsistent. Sometimes it gets stuck at an accuracy of 50% and other times it correctly trains without changing anything. Thus, it was important to run changes multiple times to get an accurate estimate of the true error of the network. Just by running the network for 25 epochs with a learning rate of 0.075, the needed accuracy was achieved.

<h4> 2.4.4 Does the test performance reach a given baseline 68% performance? (Please include a screenshot of Kaggle Submission) </h4> <p>
    
Yes, both models passed the baseline. 

![Basic solution with 78%](basic.png "Basic solution with 78%")


<h2>Part 3: Creative Solution</h2><p>

<h3>3.1 Open-ended Code:</h3><p>
You may follow the steps in part 2 again but making innovative changes like creating new features, using new training algorithms, etc. Make sure you explain everything clearly in part 3.2. Note that reaching the 75% creative baseline is only a small portion of this part. Any creative ideas will receive most points as long as they are reasonable and clearly explained.

In [ ]:
graph = clean(pd.read_csv('./graph.csv', sep=',',header=None, encoding='utf8'))

In [ ]:
def get_regions(runs, depth, partition):
    """
    Returns a set of geographic regions as a dictionary
    Each region is grown as a tree of fixed depth in the counties graph
    Can either partition the US map, or allow overlaps between regions 
    Each run covers the whole US
    """
    regions = {}
    l = 0

    for j in range(runs):
        remaining = set(graph['SRC'].unique())
        
        while(True):

            center = np.random.choice(list(remaining))
            neighbors = [center]

            for i in range(depth):
                
                if(partition):
                    neighbors += graph[
                        (graph['SRC'].isin(neighbors)) & (~graph['DST'].isin(neighbors)) & (graph['DST'].isin(remaining))
                    ]['DST'].unique().tolist()

                    neighbors += graph[
                        (graph['DST'].isin(neighbors)) & (~graph['SRC'].isin(neighbors)) & (graph['SRC'].isin(remaining))
                    ]['SRC'].unique().tolist()
                    
                else:
                    neighbors += graph[
                        (graph['SRC'].isin(neighbors)) & (~graph['DST'].isin(neighbors))
                    ]['DST'].unique().tolist()

                    neighbors += graph[
                        (graph['DST'].isin(neighbors)) & (~graph['SRC'].isin(neighbors))
                    ]['SRC'].unique().tolist()
                    
            regions[str(len(regions))] = neighbors

            remaining = remaining - set(neighbors)

            if(len(remaining) == 0):
                break

        l = len(regions)

    return regions

In [ ]:
# Merge regions from multiple runs
small_regions = get_regions(1,4,True)
medium_regions = get_regions(1,6,True)
large_regions = get_regions(2,8,True)
xl_regions = get_regions(2,15,True)

l = len(small_regions)

for r,c in large_regions.items():
    small_regions[r + 'large'] = c

for r,c in medium_regions.items():
    small_regions[r + 'medium'] = c
    
#for r,c in xl_regions.items():
#    large_regions[r + 'medium'] = c

#regions = small_regions
regions = large_regions

print(len(regions))

In [ ]:
def addCreativeFeats(df):
    
    # Adding a population change feature 
    df['Pop Change Rate'] = df.apply(lambda row: row['MigraRate'] + row['BirthRate'] - row['DeathRate'],axis=1)

    norms = df[['Pop Change Rate']]
    df['PopNorm'] = norms.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
    
    # Adding a binary feature for each state
    def addStateFeat(df,state):
        df[state] = df.apply(lambda row: int(row['County'].split(', ')[-1] == state) * 1,axis=1)
        
    for state in states:
        addStateFeat(df,state)
        
    # Adding a binary feature for each region
    def addRegionFeat(df,region):
        df[region] = df.apply(lambda row: int(row['FIPS'] in regions[region]) * 1,axis=1)
        
    for region in regions:
        addRegionFeat(df,region)
        
    return df

In [ ]:
def getStates(df):
    # helper function to get all states
    return list(set(c.split(', ')[-1] for c in df['County'].tolist()[:-1]))

<h4> Soft-Margin SVM </h4>

In [ ]:
svmdf = preprocess(trainfile, True)
svmdf = svmdf.sample(frac=1).reset_index(drop=True)
states = getStates(svmdf)
svmdf = addCreativeFeats(svmdf)

In [ ]:
features = ['MedianIncome', 'MigraRate', 'BirthRate', 'BachelorRate', 'UnemploymentRate', 
            'DeathRate'] + states + list(regions.keys())

normfeats = ['MedianIncome', 'MigraRate', 'BirthRate', 'BachelorRate', 'UnemploymentRate', 'DeathRate']

label = 'Label'

cs = np.logspace(0,1,num=20)
#cs = [26]

kernels = ['rbf']

k = 10

creative_svm = train_svm(svmdf, features, k, normfeats, label, cs, kernels)

In [ ]:
print('(Creative SVM)')
print('k-fold cross-val accuracy: ' + str(creative_svm[0]))
print('best kernel: ' + creative_svm[2])
print('best C: ' + str(creative_svm[3]))

In [ ]:
all_feats = svmdf[features].copy()
normalize(all_feats, normfeats, 'std')
all_feats = all_feats.to_numpy()
all_labels = svmdf[label].to_numpy()

model = creative_svm[1]

trained_model = model(all_feats,all_labels)
preds = trained_model(all_feats)

acc = weighted_accuracy(preds, all_labels)

print('SVM training accuracy: ' + str(acc))

<h4> Neural Network </h4>

In [ ]:
def addStateFeats(df):
    # adds all states to df
    states = getStates(df) # get all states
    def addStateFeat(df,state):
        df[state] = df.apply(lambda row: int(row['County'].split(', ')[-1] == state),axis=1)
    for state in states:
        addStateFeat(df,state)
    if 'DC' not in df.columns:
        # if no DC, add 0's, only needed for no_label.csv
        df['DC'] = pd.Series([0 for x in range(len(df.index))], index=df.index)

In [ ]:
len(['MedianIncome', 'MigraRate', 'BirthRate', 'BachelorRate', 'DeathRate', 'UnemploymentRate', 
            ] + list(regions.keys()) + states)

In [ ]:
# Modified from the discussion notebook
class CSVDataset(Dataset):
    def __init__(self, dataset, train):
        # Where the initial logic happens like reading a csv, doing data augmentation, etc.
        self.dataset = dataset   # df is a Pandas dataframe in the above format
        self.train = train  # train is True when there are DOM/GOP columns, False otherwise

    def __len__(self):
        # Returns count of samples (an integer) you have. 
        return len(self.dataset.index)

    def __getitem__(self, idx):
        # Given an index, returns the correponding datapoint. 
        # This function is called from dataloader like this:
        # img, label = CSVDataset.__getitem__(99)  # For 99th item
        # sol = 'basic' when using basic solution, 'creative' when using creative
        result = self.dataset.iloc[idx]
        #features = ['MedianIncome', 'MigraRate',
    #'BirthRate', 'DeathRate', 'BachelorRate', 'UnemploymentRate', 'TX', 'ND', 'WY', 'NH', 'AL', 'NE', 'RI', 'KS', 'SC', 'IL', 'NJ', 'OR', 'NV', 'DC', 'CT', 'IN', 'MI', 'VT', 'MO', 'VA', 'LA', 'HI', 'OK', 'MD', 'KY', 'WA', 'ID', 'AZ', 'MS', 'DE', 'FL', 'CA', 'NY', 'WI', 'WV', 'ME', 'NC', 'MT', 'MA', 'PA', 'MN', 'SD', 'CO', 'AR', 'TN', 'OH', 'NM', 'IA', 'UT', 'GA']
        
        features = ['MedianIncome', 'MigraRate', 'BirthRate', 'BachelorRate', 'DeathRate', 'UnemploymentRate', 
             
                   ] + list(regions.keys()) + states
        
        if self.train:
            return (torch.Tensor(result[features]), torch.as_tensor(result['Label']))
        else:
            return torch.Tensor(result[features])

# Modified from the discussion notebook
class Net(nn.Module):
    def __init__(self):
        """
        You need to initialize most NN layers here.
        """
        super(Net, self).__init__()
        # change first number to match inputs
        # according to resources, hidden layer <= 2 * input
        self.fc1 = nn.Linear(len(features), 20)
        self.fc2 = nn.Linear(20, 20) # hidden layer
        self.fc5 = nn.Linear(20, 1)

    def forward(self, x):
        """
        Define in what order the input x is forwarded through all the NN layers to become a final output. 
        """
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #x = F.relu(self.fc3(x))
        #x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        return x

In [ ]:
def trainNN(xTr, epochs=10, lr = 0.06):
    """
   returns new neural networks after epochs with a learning rate of lr
    """
    net = Net()     # new net
    criterion = nn.MSELoss()    # loss function
    optimizer = optim.SGD(net.parameters(), lr = lr)    # optimization
    dataset = CSVDataset(xTr, True)     # dataset for PyTorth
    dataloader = DataLoader(dataset, shuffle = True) # shuffle data
    networks = []   # all networks
    accuracies = []     # all accuracies
    for epoch in range(epochs):
        preds = []
        real = []
        for batch_idx, (input, target) in enumerate(dataloader):
            optimizer.zero_grad() # resets gradient
            output = net(input) # output for current input
            loss = criterion(output, target.float()) # calculate loss
            loss.backward() # backprop?
            optimizer.step()
            prediction = output.item() # prediction with current input
            preds += [1 if prediction >= 0.2 else 0]   # 0/1 prediction with threshold
            real += [target.item()]    # real label
        #print('preds: ' + str(preds))
        #print('real: ' + str(real))
        accuracy = weighted_accuracy(preds, real) # calculate accuracy
        networks += [net]
        accuracies += [accuracy]
        print("epoch = " + str(epoch) + ", loss = " + str(loss)) # loss for current epoch
        print(accuracy) # prints out Tensor?? not sure why, should be a float
    net = networks[np.argmax(accuracies)] # return best network based on training accuracy
    return net

def predictNN(net, xTe):
    preds = []
    real = []
    raw = []
    dataset = CSVDataset(xTe, True) # dataset for PyTorch
    dataloader = DataLoader(dataset, shuffle = False)
    for x, y in dataloader:
        prediction = net(x).item()  # prediction with network
        raw += [prediction] # raw predictions
        prediction = 1 if prediction >= 0.2 else 0 # 0/1 prediction with threshold
        y = y.item()    # real label
        preds += [prediction]
        real += [y]
    print(preds) # all 0/1 predictions
    print(real) # all real labels
    print(raw) # all raw predictions
    return weighted_accuracy(preds, real)

In [ ]:
df = preprocess(trainfile, True) # create Pandas dataframe
df = addCreativeFeats(df) # Add creative features
xTr, xTe = split(df, proportion=0.80, norm='std') # split into training and test

In [ ]:
net = trainNN(xTr, 15, 0.0075) # train network 
accuracy = predictNN(net, xTe) # calculate validation accuracy
print(accuracy)

<h2>Part 4: Kaggle Submission</h2><p>
You need to generate a prediction CSV using the following cell from your trained model and submit the direct output of your code to Kaggle. The CSV shall contain TWO column named exactly "FIPS" and "Result" and 1555 total rows excluding the column names, "FIPS" column shall contain FIPS of counties with same order as in the test_2016_no_label.csv while "Result" column shall contain the 0 or 1 predictions for corresponding columns. A sample submission file can be downloaded from Kaggle.

In [ ]:
def submission_nn(net):
    """
    creates 'submission.csv' with required format from predictions using net on df
    can be easily modified to be used for non-PyTorch stuff
    """
    df = preprocess(testfile, False) # create Pandas dataframe
    #addStateFeats(df) # adds state features
    
    addCreativeFeats(df)
    #addChangeFeats(df, preprocess(TEST_FILE2, False))
    features = ['MedianIncome', 'MigraRate', 'BirthRate', 'DeathRate', 'BachelorRate', 'UnemploymentRate']
    normalize(df, features, 'std')  # renormalize
    dataset = CSVDataset(df, False) # dataload for PyTorch
    dataloader = DataLoader(dataset, shuffle = False)
    result = []
    for x in dataloader:
        prediction = net(x).item() # predict using network
        prediction = 1 if prediction >= 0.2 else 0 # 0/1 prediction with threshold
        result += [prediction]
    for index, fips, in enumerate(df['FIPS']):
        # add FIPS id to prediction
        result[index] = [fips] + [result[index]]
    dfSubmission = pd.DataFrame(result)     # create Pandas dataframe
    dfSubmission.columns = ['FIPS', 'Result']   # rename columns
    dfSubmission.to_csv('submission.csv', index = False)    # output file

In [ ]:
def submission_svm(df, svm):
    all_feats = normalize(df[features].copy(), normfeats, 'std').to_numpy()
    all_labels = df[label].to_numpy()

    trained_model = svm(all_feats,all_labels)

    kaggleset = normalize(addCreativeFeats(preprocess(testfile, False).copy()),normfeats,'std')

    testfeats = kaggleset[features].to_numpy()

    preds = trained_model(testfeats).astype(int)

    dfout = pd.DataFrame(data=preds, columns=["Result"])
    dfout.index=kaggleset.index
    dfout.insert(0,"FIPS",pd.Series(kaggleset['FIPS']))
    #print(dfout)
    dfout.to_csv('./Results.csv',index=False)

In [ ]:
submission_svm(svmdf, creative_svm[1])

In [ ]:
submission_nn(net)

<h3>3.2 Explanation in Words:</h3><p>

<h4> 3.2.1 How much did you manage to improve performance on the test set compared to part 2? Did you reach the 75% accuracy for the test in Kaggle? (Please include a screenshot of Kaggle Submission) </h4>

Our NN reached 86% accuracy, and our SVM reached 82% accuracy on the Kaggle test set. 

![Creative with 83% accuracy](creative.png "Creative with 83% accuracy")



<h4> 3.2.2 Please explain in detail how you achieved this and what you did specifically and why you tried this. </h4>

After iterative improvements, both our SVM and neural network models were able to achieve over 80% accuracy on the public Kaggle test set. 

We achieved an accuracy of 83% on the creative test by implementing a neural network using the PyTorch library. 
From initial testing, the neural network seemed to do well using only the 6  given features ('MigraRate', 'BirthRate', 'DeathRate', 'BachelorRate', 'UnemploymentRate') using a min-max normalization. The network uses the mean squared error for the loss function and the stochastic gradient descent for the optimization.
The network initially had the 6 input nodes which were fully connected to 12 nodes in a single hidden layer. These nodes used a linear function with a ReLU activation. The single hidden layer was then fully connected to a single output node. 12 nodes were chosen, because based on the resources we read, the maximum number of nodes in the hidden layer should be less than or equal to the number of input nodes.

To train the network, after preprocessing the dataset, the dataset was shuffled and split into a training and a validation set. The two sets are then independently normalized to not have any data leakage. The training set is used to train the network by first initializing a PyTorch Net and then training the same network over and over again for a number of epochs. For each epoch, the accuracy of the prediction is saved. When the training is done, then the network with the highest training accuracy is chosen and then used to predict the validation set. All accuracies are calculated using the given weighted_accuracy() function. 

The first parameter that was varied was the learning rate. This was manually adjusted based on each new training iteration, because the accuracy of the network varies a lot even without changing anything. Therefore, the accuracy of each iteration was judged manually when determining what value was better. A learning rate of 0.0075 was selected.
The linear nodes and ReLU activation functions were not varied, because based on prior knowledge of historical voting habits and election results, it was determined that the dataset would be relatively easy to separate and label using the current neural network. The size and shape of the network was also adjusted, but the accuracy always became worse so the shape was kept at a single hidden layer with two times the input nodes.

The next step was to change the normalization. Instead of using a min-max normalization, we decided to use a standardization normalization, because it is less likely to be affected by outliers. This increased the validation accuracy by an appreciable amount, because the accuracy seemed to have stopped improving before. The learning rate was then adjusted for the change. This changed the validation accuracy from the high 70s% to the low 80s%.

Next, the accuracy had hit another wall. We then decided to add new features. The first set of features we added were indicator features as to what state the county is located in. We chose this to be the first feature to add, because counties in the same state are likely to have similar political views, except for a few outliers that have large cities. This was added by looking at the county name to see the state abbreviation and setting that state to 1 and all other states to 0. After adding the feature, the network size was adjusted to match the size of the input (56 input features and 112 nodes in the single hidden layer). This increased the accuracy from the low 80s% to the high 80s%.
With these simple changes, we were able to surpass the threshold accuracy for the creative.

Since we worked as a team, we were able to diversify our efforts, so at the same time as the neural network was being improved the SVM was too. The SVM served as a testing ground for introducing more different types of features. The most successful features were derived from graph.csv, which describes a graph where the nodes are the counties, and two counties are connected by an edge if they border each other. For the same reason that we expect election results to correlate with the state, we expect them to more generally correlate with geography on all scales. Looking at past election maps, patterns emerge on a large scale as different regions in the US tend to share in broad cultural identity, and on a small scale as cultural hubs like large cities or college towns can alter the local political landscape. Thus for each country we would like to get an idea of which counties are close by, and which large-scale region(s) it may belong to. We extract this data from the graph by growing forests of trees; we call each tree a region. To identify large-scale regions, we repeatedly grow forests of trees starting at random nodes in the graph with a large fixed depth that cover the U.S., dividing it in multiple ways into regions containing hundreds of counties, with the hope that a subset of them are shaped in such a way to capture the dynamics of the real-world U.S. Similarly for small-scale regions, we partition the U.S. just once into trees of a small fixed depth. Since these regions are not constrained by state borders, they are more expressive than the states. We add binary features for each region where 1 means the county belongs to the region, and 0 means it does not. We trained the SVM with these new features, which total around 250, and saw an improvement of over 5%, surpassing 80% on the Kaggle set.

We then gave the neural network these features as well, but it took some effort to get it to work well with them. For one, we could not keep the 1:2 ratio between the size of the input layer and the size of the hidden layer. So many nodes cause significant overfitting after only a few training epochs. We eventually settled on the hidden layer having 20 nodes. After some more improvements on the types of regions we turn into features, our neural network exceeded 86% accuracy on the Kaggle test set.

Of course, we also tried things which did not work. 

One idea we had for using graph.csv was to ‘blur’ the results of neighboring or close-by counties. That would mean taking weighted averages of the predictions made by a learning algorithm over a region surrounding each county. The main problem with this approach was the sparseness of the training set provided. Because there are many more counties in the US than are in the training set, many counties had either no or very few direct neighbors in the training set. In order to get enough neighbors we had to consider counties up to 3 borders away (that is, separated by 2 intermediate counties), and this did not improve our accuracy. 

We also thought that we might be able to combine multiple simpler algorithms into an ensemble model which returns an average of their predictions. We hoped that this might combat overfitting, and also make up weaknesses in individual algorithms. However, no ensemble of 2-6 learning algorithms (including the SVM) was able to out-do the SVM on its own. Here are some of the algorithms we tried.

We tried using ridge regression so that at least one of the algorithms in the ensemble would take into account the quantitative election results in each county; that is, the vote totals. The ‘labels’ we ask ridge regression to predict is the percentage of votes which go to DEM. This encodes the degree to which the election swung in one direction or the other while also normalizing the data without having to do any additional processing. 

We tried to use a decision tree because although it is prone to overfitting, we predicted that when put into an ensemble with methods less likely to overfit it may strengthen the accuracy. However, no combination of hyperparameters seemed to lead to a model which did better than the SVM. 

We also tried some algorithms which we hypothesized would not do well, just to make sure that our thinking was not off. For example we tried using naive bayes, which very strongly relies on the i.i.d assumption which we discuss earlier and thought would not apply. We were correct, as we could not get naive bayes to even pass the basic 68% threshold. 


<h2>Part 5: Resources and Literature Used</h2><p>

Week 11 Discussion notebook

https://pythonprogramming.net/introduction-deep-learning-neural-network-pytorch/

https://www.heatonresearch.com/2017/06/01/hidden-layers.html
